# Thalamic nuclei mean kurtosis estimation

> Import modules

In [1]:
import pandas as pd
import os
import re
import numpy as np
import nibabel as nb
import argparse
import textwrap

> Set input subjects / file locations

In [2]:
dataLoc = os.getcwd()
subjects = [x for x in os.listdir(dataLoc) if x.startswith('N') or x.startswith('F')]

In [3]:
def get_segment_map(subject,side):
    '''
    Returns nibabel matrix of 
    the connectivity-based segmentation map of thalamus
    '''
    segment_file = os.path.join(dataLoc,
                              subject,
                              'segmentation',
                              side,
                              'dki_biggest.nii.gz')
    f = nb.load(segment_file)
    segment_map = f.get_data()
    return segment_map

In [4]:
def get_mk_map(subject):
    '''
    Returns nibabel matrix of
    the whole brain mean kurtosis image
    '''
    mk_file = os.path.join(dataLoc,
                         subject,
                         'DKI',
                         'kmean.nii')
    f = nb.load(mk_file)
    mk_map = f.get_data()
    return mk_map

In [5]:
def get_mean_MK_for_nuclei(segment_map, mk_map, cortex):
    '''
    Returns mean of the MK, and the volume 
    of the thalamic segment
    in connection to sepcific cortex
    '''
    nuclei_dict = {"LPFC":1,
                  "LTC":2,
                  "MPFC":3,
                  "MTC":4,
                  "OCC":5,
                  "OFC":6,
                  "PC":7,
                  "SMC":8}
    nucleiNum = nuclei_dict[cortex]
    cortexMap = segment_map == nucleiNum # returns boolean matrix of the matching cortex
    segment_volume = np.sum(cortexMap) # count number of True (voxels of the matching cortex)
    
    segment_mk = cortexMap * mk_map # returns mk_map within the thalamic nucleus of matching cortex
    mk_mean = segment_mk[segment_mk!=0].mean() # non-zero mean

    return mk_mean, segment_volume

In [27]:
def get_short_side(side):
    if side == 'left':
        side_s = 'lh'
    else:
        side_s = 'rh'
    return side_s

In [28]:
def get_cortex_volume(subject, side, cortex):
    '''
    Returns the volume of the given cortex ROI
    '''
    side_s = get_short_side(side)
    cortexROI = os.path.join(dataLoc,
                            subject,
                            'ROI',
                            '{side_s}_{cortex}.nii.gz'.format(side_s=side_s,
                                                             cortex=cortex))
    cortexVolume = get_volume(cortexROI)
    return cortexVolume

In [29]:
def get_connectivity_for_nuclei(subject, side, cortex):
    '''
    Returns the sum of connectivity
    in the thalamic segment
    '''

    
    side_s = get_short_side(side)
    connectivity_file = os.path.join(dataLoc, 
                                    subject,
                                    'segmentation',
                                    side,
                                    'seeds_to_{side_s}_{cortex}.nii.gz'.format(side_s=side_s,
                                                                                cortex=cortex))
    totalConnectivity = get_volume(connectivity_file)
    return totalConnectivity
    

In [30]:
def get_volume(imgFile):
    return nb.load(imgFile).get_data().sum()

In [33]:
def thalamic_mk(subject, side):
    '''
    Returns a pandas dataframe,
    containing MK of thalamic segments 
    for a given subject
    '''
    group = subject[:3]
    segment_map = get_segment_map(subject, side)
    mk_map = get_mk_map(subject)
    
    df = pd.DataFrame()
    for cortex in ["LPFC", "LTC", "MPFC", "MTC", "OCC", "OFC", "PC", "SMC"]:
        totalConnectivity = get_connectivity_for_nuclei(subject, side, cortex)
        cortexVolume = get_cortex_volume(subject, side, cortex)
        mk_mean, segment_volume = get_mean_MK_for_nuclei(segment_map, mk_map, cortex)
        
        cortex_df = pd.DataFrame(data = [[mk_mean, 
                                          segment_volume,
                                          totalConnectivity,
                                          cortexVolume,
                                          cortex, 
                                          side, 
                                          group]], 
                                 index=[subject], 
                                 columns=['MK',
                                          'nucleiVolume',
                                          'totalConnectivity',
                                          'cortexVolume',
                                          'cortex',
                                          'side',
                                          'group'])
        df = pd.concat([df, cortex_df])
    
    df['relativeConnectivity'] = df.totalConnectivity / df.totalConnectivity.sum()
    df['thalamucVolume'] = get_
    return df

In [34]:
thalamic_mk('NOR76_KKT','left')

,MK,nucleiVolume,totalConnectivity,cortexVolume,cortex,side,group,relativeConnectivity
NOR76_KKT,0.760467946529,104,2460381,28697,LPFC,left,NOR,0.146904
NOR76_KKT,0.947559952736,5,351841,51218,LTC,left,NOR,0.021008
NOR76_KKT,0.907443940639,88,2146572,32587,MPFC,left,NOR,0.128167
NOR76_KKT,0.716338574886,163,1122250,15056,MTC,left,NOR,0.067007
NOR76_KKT,nan,0,22813,27465,OCC,left,NOR,0.001362
NOR76_KKT,1.03520548344,37,2044323,15889,OFC,left,NOR,0.122062
NOR76_KKT,1.02211666107,326,6621119,62184,PC,left,NOR,0.395333
NOR76_KKT,0.828021883965,4,1978898,36532,SMC,left,NOR,0.118156


In [42]:
def main():
    '''
    Estimates the mean kurtosis in the thalamic segments
    for all subjects.
    The thalamic segments is obtained from the connectivity-based segmentation
    from FSL.
    
    It returns mk_data.csv containing all subjects' data.
    
    '''
    allData = pd.DataFrame()
    for subject in subjects:
        for side in ['left','right']:
            subjectDf = thalamic_mk(subject, side)
            allData = pd.concat([allData, subjectDf])
    
    allData.to_csv('mk_data.csv')

```
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description=textwrap.dedent('''\
            {codeName} : Estimates mean kurtosis in the thalamic nuclei
            ===========================================================
            '''.format(codeName=os.path.basename(__file__))))

    parser.add_argument(
        '-s', '--subjects',
        help='subject list',
        nargs='+')

    parser.add_argument(
        '-side', '--side',
        help='side')
    
    args = parser.parse_args()    
    thalamic_mk(args)
```